In [1]:
#полный пайплайн TEA
#изготавливает модель, сохраняет, разбирает, генерит с помощью модели

In [2]:
install = False
#Через requirements поставить это затруднительно, последовательность установки важна, так как часть либ компилируются
#Эта последовательность установки, как бы странно она ни выглядела, работает. Не в любых окружениях, но во многих.
if install:

    !pip install deepspeed==0.14.4

    !pip install --upgrade pip
    !pip install --upgrade transformers==4.46.2 tyro==0.9.8 triton==2.3.1 trl==0.12.0

    !pip install -v "xformers==0.0.29.post1"
    #!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps -v "peft==0.13.2"
    !pip install --no-deps packaging ninja einops flash-attn bitsandbytes==0.44.1
    !pip install accelerate==0.34.2
    !pip uninstall peft -y
    !pip install -v "peft==0.13.2"

In [3]:
import sys
import time
import json
import os
from collections import OrderedDict
from pathlib import Path
from collections import deque
import os, pickle, random
import psutil
import warnings
import codecs
import numpy as np
import pandas as pd
import requests
from typing import Dict, List, Optional
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
from peft import PeftModel, PeftConfig, PeftModelForCausalLM

import ensembles
from sequential_models import GPTAdapterLayer

In [4]:
current_device = torch.cuda.current_device()
print(f"Используется GPU с номером: {current_device}")

Используется GPU с номером: 0


In [5]:
device = torch.device("cuda")

In [6]:
# Проверяем доступность CUDA и количество GPU
#если надо, можем переключиться на другую GPU
if 0:
    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:  # Если есть более 1 GPU
            device = torch.device("cuda:1")  # Явно указываем устройство 1
            print(f"Переключились на GPU 1")
        else:
            print("Доступен только один GPU (номер 0), переключение невозможно")
    else:
        print("CUDA недоступно")

In [7]:
start_train = False#Запускаем ли мы сейчас обучение с нуля, или с чекпоинта
train_transformer = True#надо ли обучать трансформерный адаптер
train_lm_head = True#надо ли обучать lm_head
learnable_linear_model = True#учим ли мы линейную субмодель внутри lm_head


cardinality = 128256#размер словаря токенов

padding_token = 128009
forbidden_tokens_list = [padding_token]
seed = int(np.random.rand() * 1000000)#random seed for data sampling

bits_per_number = 4

In [8]:
max_batch_size4loss = 1024 * 2#loss считаем батчами, во избежание out of memory error
head_max_batch_size = 1024#lm_head заускаем батчами, во избежание out of memory error

In [9]:
#конфигурации моделек
mode = 'pretrain'#'finetune', 'pretrain'
model_version = "heavy_2"
transformer_adapter_params = {}
lm_head_adapter_params = {'cardinality':cardinality}
optimizer_params = {}
if model_version == "heavy_2":
    transformer_adapter_params['embed_dim'] = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    #трансформерный адаптер
    transformer_adapter_params['num_heads_tlayer'] = 16
    transformer_adapter_params['ff_dim'] = transformer_adapter_params['embed_dim']  * 4
    transformer_adapter_params['t_layers_count'] = 2
    transformer_adapter_params['layer_configs'] = [1024*4] * 2
    optimizer_params['batch_size'] = 1#почему-то распараллеливание ухудшает скорость, а так щдесь можно поставить и другое число
    optimizer_params['accum_batch'] = 900
    optimizer_params['lr'] = 1e-5
    transformer_adapter_params['transformer_adapter_weight'] = 1e-3#идея в том, что со старта адаптерный слой должен влиять совсем чуть-чуть
    transformer_adapter_params['dropout'] = 0.1
    optimizer_params['opt_type'] = 'adam'

    #адаптер для LM-head
    lm_head_adapter_params['embedding_size'] = transformer_adapter_params['embed_dim']
    lm_head_adapter_params['heavy_head'] = True
    lm_head_adapter_params['net_dropout_rate'] = 0
    lm_head_adapter_params['individ_dropout_rate'] = 0.03
    lm_head_adapter_params['layer_configs_head'] = [2048] * 5 + [1024] * 5
    lm_head_adapter_params['sample_features'] = 1
    lm_head_adapter_params['composition_size'] = 1
    lm_head_adapter_params['memnet_params'] = {}
    lm_head_adapter_params['use_memnets'] = False

elif model_version == "heavy_6":
    transformer_adapter_params['embed_dim'] = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    #трансформерный адаптер
    transformer_adapter_params['num_heads_tlayer'] = 16
    transformer_adapter_params['ff_dim'] = transformer_adapter_params['embed_dim']  * 4
    transformer_adapter_params['t_layers_count'] = 2
    transformer_adapter_params['layer_configs'] = [1024*4] * 2
    optimizer_params['batch_size'] = 1#почему-то распараллеливание ухудшает скорость, а так щдесь можно поставить и другое число
    optimizer_params['accum_batch'] = 9
    optimizer_params['lr'] = 1e-5
    transformer_adapter_params['transformer_adapter_weight'] = 1e-3#идея в том, что со старта адаптерный слой должен влиять совсем чуть-чуть
    transformer_adapter_params['dropout'] = 0.1
    optimizer_params['opt_type'] = 'adam'

    #адаптер для LM-head
    lm_head_adapter_params['embedding_size'] = transformer_adapter_params['embed_dim']
    lm_head_adapter_params['heavy_head'] = True
    lm_head_adapter_params['net_dropout_rate'] = 0
    lm_head_adapter_params['individ_dropout_rate'] = 0.03
    lm_head_adapter_params['layer_configs_head'] = [2048] * 5 + [1024] * 5
    lm_head_adapter_params['sample_features'] = 1
    lm_head_adapter_params['composition_size'] = 1
    lm_head_adapter_params['memnet_params'] = {}
    lm_head_adapter_params['use_memnets'] = False

In [10]:
lm_head_adapter_params['head_max_batch_size'] = head_max_batch_size
lm_head_adapter_params['learnable_linear_model'] = learnable_linear_model

In [11]:
path2tadapter = f"tadapter_{model_version}.pth"
path2lmhead = f"lmhead_{model_version}.pth"
tokenizer_name = model_name

In [12]:
#настройки датасета

if mode == 'pretrain':
    #претрейн
    max_seq_len = 6900
    max_seq_len4inference = 6850
else:
    #дообучение
    max_seq_len = 2050
    max_seq_len4inference = 2000

In [13]:
#Загрузили исходную модельку
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")#, cache_dir="D:\cache\huggingface\\"+ model_name)

if bits_per_number == 4:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"#, bnb_4bit_compute_dtype=torch.bfloat16
    )
elif bits_per_number == 8:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True, bnb_8bit_use_double_quant=True, bnb_8bit_quant_type="nf8"#, bnb_8bit_compute_dtype=torch.bfloat16
    )
else:
    bnb_config = None

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
              model_name,
              quantization_config=bnb_config,
              cache_dir="D:\cache\huggingface\\"+ model_name)


<>:20: SyntaxWarning: invalid escape sequence '\c'
<>:20: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_367795/3520330652.py:20: SyntaxWarning: invalid escape sequence '\c'
  cache_dir="D:\cache\huggingface\\"+ model_name)
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/ssdovgan/main/lib/python3.12/site-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [14]:
#model.lm_head
torch.save(model.lm_head.weight, "lin_model.pth")

In [15]:
class InstructDatasetR(Dataset):
    def __init__(self, data_file, tokenizer, max_seq_length, cut=None):
        self.data_file = data_file
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.cut = cut
        self.data = self.load_data()
        self.log_samples = deque(maxlen=45)

    def load_data(self):
        with open(self.data_file, 'rb') as f:
            data = pickle.load(f)
        random.seed(seed)
        data = random.sample(data, len(data))
        # if self.cut is None:
        #     data = [data_cur for data_cur in data if len(data_cur[1])>5]
        # else:
        #     data = [data_cur for data_cur in data[:self.cut] if self.cut>5]
        # for i in range(100):
        #     print('***', data[i])
        #     try:
        #         if (not 'Алис' in data[i][0]) and (not 'Элеон"' in data[i][0]) and (not 'Софи"' in data[i][0]) and (not 'Сэм"' in data[i][0]) (not 'Алис' in data[i][1]) and (not 'Элеон"' in data[i][1]) and (not 'Софи"' in data[i][1]) and (not 'Сэм"' in data[i][1]):
        #             print('***', data[i])
        #     except Exception:
        #         pass
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.log_samples.append(self.data[idx])
        parts = self.data[idx]
        text = parts[0]
        label = parts[1]
        parts = [parts[0], parts[1], 1]
        for r_variant in [-2,-1,-0.5,0.5,1,2]:
            s = f"<r{r_variant}>"
            if s in label:
                parts[-1] = r_variant
                label = label.replace(s, '')
                break
        r = parts[-1]
        if text is None:
            text = label
        
        # Кодируем текст и метку с помощью tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        
        # Кодируем метку
        label_encoding = self.tokenizer.encode_plus(
            label,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        label_ids = label_encoding['input_ids']
        return {
            'input_ids': input_ids[0],
            'attention_mask': attention_mask,
            'labels': label_ids[0],
            'mult': torch.tensor(r)
        }

In [16]:
# Создание Dataset
#такая конструкция нужна на случай, что вы загружаете датасет на диск, сейчас час ночи, 
#датасет догрузится примерно через час, а завтра вам рано вставать
#с таким подходм код будет выполняться, когда найдёт на диске датасет
while 1:
    try:
        if mode == 'pretrain':
            #dataset = InstructDatasetR("./data/dataset_llm_full_mflat.pkl", tokenizer, max_seq_len, cut=None)
            dataset = InstructDatasetR("../data_tea/dataset_llm_full_flat.pkl", tokenizer, max_seq_len, cut=None)
        else:
            dataset = InstructDatasetR("../data_tea/dataset_llm_full_instruct.pkl", tokenizer, max_seq_len, cut=None)
        break
    except Exception as e:
        print(e, pd.Timestamp.now())
        time.sleep(10)
len(dataset)

1071219

In [17]:
def soft_clip(tens, mn, mx):
    mn = torch.tensor(mn).to(tens.device)
    mx = torch.tensor(mx).to(tens.device)
    tens[tens<mn] = torch.nn.Sigmoid()(tens[tens<mn]) + mn
    tens[tens>mx] = torch.nn.Sigmoid()(tens[tens>mx]) + mx
    return tens
# Функция для подсчёта количества последних токенов, равных padding_token
def count_padding(tensor, padding_token):
    counts = []
    for row in tensor:
        count = 0
        for token in reversed(row):
            if token.item() == padding_token:
                count += 1
            else:
                break
        counts.append(count)
    return min(counts)
def count_trainable_parameters(model: nn.Module) -> tuple[int, int]:
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    return trainable, frozen
loss_fct = nn.CrossEntropyLoss(reduction = 'none')

In [18]:
#теперь сборка модели с адаптерами
def assemble_model(model, 
                   path2lmhead, 
                   path2tadapter, 
                   start_train,
                   to_generate,
                   lm_head_adapter_params,
                   transformer_adapter_params,
                   optimizer_params
                  ):
    if start_train:
        #инициализировать всё это дело
        torch.manual_seed(1)
        np.random.seed(1)
        random.seed(1)
        head = ensembles.EResNetPro(input_size=lm_head_adapter_params['embedding_size'], 
                   out_size=lm_head_adapter_params['cardinality'], 
                   net_dropout_rate=lm_head_adapter_params['net_dropout_rate'], 
                   individ_dropout_rate=lm_head_adapter_params['individ_dropout_rate'],
                   layer_configs=lm_head_adapter_params['layer_configs_head'], 
                   use_sigmoid_end=False, 
                   use_batchnorm=True, 
                   use_activation=True, 
                   activation=nn.LeakyReLU(), 
                   sample_features=lm_head_adapter_params['sample_features'], 
                   composition_size=lm_head_adapter_params['composition_size'], 
                   lin_bottleneck_size=None,
                   lin_model_add=nn.Linear(lm_head_adapter_params['embedding_size'], lm_head_adapter_params['cardinality']).to(device),
                   memnet_params=lm_head_adapter_params['memnet_params'],
                   use_memnets=lm_head_adapter_params['use_memnets'],
                   max_batch_size=lm_head_adapter_params['head_max_batch_size'],
                   aggregation_by_mean=False).to(device)
        head.submodels[-1].weight = torch.nn.Parameter(torch.load("lin_model.pth").to(device).to(torch.float32))
        head.submodels[-1].weight.requires_grad = lm_head_adapter_params['learnable_linear_model']
    
        tadapter = GPTAdapterLayer(initial_layer=None, 
                                 dim=transformer_adapter_params['embed_dim'], 
                                 num_heads=transformer_adapter_params['num_heads_tlayer'], 
                                 ff_dim=transformer_adapter_params['ff_dim'], 
                                 t_layers_count=transformer_adapter_params['t_layers_count'], 
                                 layer_configs=transformer_adapter_params['layer_configs'],
                                 conservativity=transformer_adapter_params['transformer_adapter_weight'],
                                 dropout=transformer_adapter_params['dropout'],
                                 composition_type='stack').to(device)
    else:
        head = torch.load(path2lmhead, weights_only=False)
        tadapter = torch.load(path2tadapter, weights_only=False)
    
    #собрать
    model.eval()

    head.to(device)
    head.by_submodels = False

    optimizer = None
    if to_generate:
        #ща сошьём
        head.half()
        tadapter.half()
        model.lm_head = head
        tadapter.initial_layer = model.model.layers[-1]
        model.model.layers[-1] = tadapter
        model.model.layers[-1].transformer_float_mode = 16
        model.model.layers[-1].lmhead_float_mode = 16
        return
    else:
        head.train()
        tadapter.train()
        if optimizer_params['opt_type'] == 'adam':
            optimizer = torch.optim.Adam([
                {'params': head.parameters(), 'lr': optimizer_params['lr']},
                {'params': tadapter.parameters(), 'lr': optimizer_params['lr'] * 0.05}
            ], lr=optimizer_params['lr'])
        else:
            momentum = 0.
            lr = optimizer_params['lr'] * 0.08
            optimizer = torch.optim.SGD([
                {'params': head.parameters(), 'lr': lr},
                {'params': tadapter.parameters(), 'lr': lr * 0.05}
            ],  momentum=momentum)
        trainable, frozen = count_trainable_parameters(head)
        print(f"Head. Обучаемые: {trainable:,}\nЗамороженные: {frozen:,}")
        trainable, frozen = count_trainable_parameters(tadapter)
        print(f"Transformer adapter. Обучаемые: {trainable:,}\nЗамороженные: {frozen:,}")
        return head, tadapter, optimizer

In [19]:
#теперь сохранение адаптеров
def save_adapters(head, tadapter, path2lmhead, path2tadapter):
    head.training = False
    torch.save(head, path2lmhead)
    torch.save(tadapter, path2tadapter)
    head.training = True

In [20]:
head, tadapter, optimizer = assemble_model(model, 
                path2lmhead, 
                path2tadapter, 
                start_train=start_train,
                to_generate=False,
                lm_head_adapter_params=lm_head_adapter_params,
                transformer_adapter_params=transformer_adapter_params,
                optimizer_params=optimizer_params
              )

use_memnets False


/tmp/ipykernel_367795/2452345961.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  head.submodels[-1].weight = torch.nn.Parameter(torch.load("lin_model.pth").to(device).t

use_memnets True
use_memnets True
Head. Обучаемые: 1,410,552,156
Замороженные: 0
Transformer adapter. Обучаемые: 701,224,480
Замороженные: 0


In [21]:
proba_drop_idx = 0.04

In [ ]:
#TRAIN!!!
#Если вы задаётесь вопросом, почему такой странный цикл обучения, давайте посмотрим, что не сработало в "нормальных" циклах обучения.
#Можно было запихнуть трансформерный адаптер и lm-resnet в исходную LLM, разморозить их и заморозить всё остальное. 
#НО тогда по непонятной причине выходила крайне плохая точность обучения. loss = 1.65 на датасете, где у классического TEA (ну, чисто резнеты) loss = 0.85
#Две недели поисков выявили, что видимое различие в том, что в одном случае адаптер обучает ВНУТРИ сетки, а в другом СНАРУЖИ. Снаружи ПОЧЕМУ-ТО лучше
#
#Если проинференсить трансформерный адаптер, а потом LM-head, то запросто может получиться, что на этапе backward мы сдохнем по памяти.
#Как фиксить? Делаем, чтобы для всего множества эмбеддингов от трансформерной части loss считался батчами
#В таком случае возникает ошибка, мол, чё нам делать с градиентами в трансформерном адаптере - мы же их уже посчитали, а ты предлагаешь считать ещё раз!
#Решаем так: берёт сабсемпл от эмбеддингов трансформерного адаптера и считаем loss на сабсемпле
#Это не вполне честно, так как мы менее тщательно учитываем длинные последовательности, но эй, это работает!

print('batch_size', optimizer_params['batch_size'])
i_pointer = 0

batch_accum_counter = 0
loss_array = []
metr_array = []
while 1: 
    input_ids_full = []
    labels_full = []
    weights_full = []
    i = 0
    offset_counter = 0
    while i < optimizer_params['batch_size']:
        offset_counter += 1
        sample_cur  = dataset[i_pointer + i]
        input_ids = sample_cur['input_ids']
        labels = sample_cur['labels']
        weights = sample_cur['mult']
        
        input_ids_len = len(input_ids)
        labels_len = len(labels)
        input_ids = input_ids[input_ids!=padding_token]
        
        drop_idx = torch.rand_like(input_ids.to(torch.float)) < proba_drop_idx
        input_ids = input_ids[~drop_idx]
        
        labels = labels[labels!=padding_token]
        padding_size = input_ids_len + labels_len - len(input_ids) - len(labels)
        input_ids_cur = torch.cat([input_ids, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        labels_cur = torch.cat([input_ids * 0 + padding_token, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        if torch.all(labels_cur) == padding_token:
            print('empty label')
            continue
        i += 1
        
        input_ids_full.append(input_ids_cur[:-1])
        labels_full.append(labels_cur[1:])
        weights_full.append(weights)

    i_pointer += i #should be batch_size
    with torch.no_grad():
        input_ids_full = torch.stack(input_ids_full).to('cuda')
        labels_full = torch.stack(labels_full).to('cuda')
        weights_full = torch.stack(weights_full).to('cuda').T

        cnt_pads = count_padding(labels_full, padding_token)
        if cnt_pads >= input_ids_full.shape[1] - 5:
            continue
        if cnt_pads > 0:
            input_ids_full = input_ids_full[:, :-cnt_pads]
            labels_full = labels_full[:, :-cnt_pads]
            
        outp = model.forward(input_ids_full, output_hidden_states=True, return_dict=True)
        outp['states'] = outp['hidden_states'][-1]
        outp['states'] = outp['states'][:, :input_ids_full.shape[1]]
        state_full = outp['states']
        weights_full2d = torch.vstack([weights_full] * labels_full.shape[-1]).to('cuda').T
        labels_cur = labels_full.ravel()#.cpu().numpy()
        weights_cur = weights_full2d.ravel()

        del labels_full
        del weights_full2d
        del weights_full
        del outp
        idx = torch.isin(labels_cur, torch.tensor(forbidden_tokens_list).to(device))
        labels_cur = labels_cur[~idx]
        weights_cur = weights_cur[~idx]
    if state_full.shape[0] == 0:
        continue

    state_full = tadapter(state_full)[0]
    state_cur = state_full.reshape([state_full.shape[0] * state_full.shape[1], state_full.shape[2]])
    state_cur = state_cur[~idx]
    del state_full
    perm = torch.randperm(state_cur.size(0), device=device)
    state_cur = state_cur[perm]
    weights_cur = weights_cur[perm]
    labels_cur = labels_cur[perm]

    loss = None
    for x_start_pointer in range(0, state_cur.shape[0], max_batch_size4loss): 
        logits = head(state_cur.to(torch.float32)[x_start_pointer: x_start_pointer + max_batch_size4loss])
        ce_loss = loss_fct(logits, labels_cur[x_start_pointer: x_start_pointer + max_batch_size4loss].view(-1))
        ce_loss = ce_loss * weights_cur[x_start_pointer: x_start_pointer + max_batch_size4loss].to('cuda')
        ce_loss = soft_clip(ce_loss, -0.6, 10.)
        loss = torch.mean(ce_loss)
        
        loss.backward()
        if state_cur.shape[0] - (x_start_pointer + max_batch_size4loss) > 0:
            print('cannot backward, shape', state_cur.shape[0] - (x_start_pointer + max_batch_size4loss))
        break
       
    del state_cur
    del weights_cur

    acc = torch.mean((torch.argmax(logits, axis=-1) == labels_cur[x_start_pointer: x_start_pointer + max_batch_size4loss]).to(torch.float16))
    del logits
    del labels_cur
    batch_accum_counter += 1
    loss_array += [torch.mean(ce_loss).item()]
    metr_array += [acc.item()]
    if batch_accum_counter >= optimizer_params['accum_batch'] :
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())
        loss_array = []
        metr_array = []
        batch_accum_counter = 0
        if (np.random.rand() < 1 and optimizer_params['accum_batch'] > 100) or (np.random.rand() < 0.1 and optimizer_params['accum_batch']  <= 100):
            try:
                save_adapters(head, tadapter, path2lmhead, path2tadapter)
                if np.random.rand()<0.1:
                    save_adapters(head, tadapter, path2lmhead + '.back', path2tadapter + '.back')
            except Exception:
                #Я не хочу, чтобы скрипт падал только потому, что кто-то завалил веь диск
                print("not saved")

batch_size 1


/tmp/ipykernel_367795/721343507.py:56: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  weights_full = torch.stack(weights_full).to('cuda').T


cannot backward, shape 991
cannot backward, shape 124


In [ ]:
#если принудительно стопнули, можно добить батч
optimizer.step()
optimizer.zero_grad()
torch.cuda.empty_cache()
print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())

In [ ]:
#ручной сейв
save_adapters(head, tadapter, path2lmhead, path2tadapter)

In [ ]:
#тестим модель
assemble_model(model, 
                path2lmhead, 
                path2tadapter, 
                start_train=False,
                to_generate=True,
                lm_head_adapter_params=lm_head_adapter_params,
                transformer_adapter_params=transformer_adapter_params,
                optimizer_params=optimizer_params
              )

prompt = 'Твоя роль: Сталин. \n Серёга: Какая фракция в игре Герои Меча и Магии 3 соответствует твоей личности? Your answer, ONLY ENGLISH (end by "|"):'
#prompt = 'Твоя роль: Девушка-флористка. \n Серёга: Какая фракция в игре Герои Меча и Магии 3 соответствует твоей личности? Your answer, ONLY ENGLISH (end by "|"):'
#prompt = 'Твоя роль: Data scientist. \n Серёга: Мне надо улучшить LLM. У меня ресурсы на запуск Llama 3.1 8B или Qwen 2.5 14B, квантованные до 4 бит. Мне нужно, чтобы мои модели былу умнее (на уровне 70 B), но при этом имели настолько же низкие требования по памяти и запускались так же быстро. Я готов обучать новую модель. Я готов пробовать другие архитектуры. Опиши план, что делать. Your answer, ONLY ENGLISH  (end by "|"):'
#prompt = 'Твоя роль: Инженер-ракетчик. \n Серёга: Мне надо сделать ракету, в домашних условиях, небольшую. Расскажи, как это провернуть? ТВОЙ ОТВЕТ НА РУССКОМ  (end by "|"):'
#prompt = 'Твоя роль: Геймер, студент-инженер. \n Серёга: Перечисли всех монстров из Doom, каких вспомнишь! Your answer, ONLY ENGLISH  (end by "|"):'
#prompt = 'Твоя роль: Гермиона Грейнджер. \n Серёга: Как справиться с василиском? Your answer, ONLY ENGLISH  (end by "|", 200 tokens):'
inputs = tokenizer(prompt, return_tensors="pt").to(device)
repetition_penalty = 1.2
t = pd.Timestamp.now()
generate_ids = model.generate(inputs.input_ids.to(device),
                                        #attention_mask=inputs["attention_mask"],
                                        max_new_tokens=200,
                                        temperature=0.01,
                                        pad_token_id=tokenizer.pad_token_id,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bos_token_id=tokenizer.bos_token_id,
                                        #no_repeat_ngram_size=3,
                                        repetition_penalty=repetition_penalty,
                                        early_stopping=True,
                                        use_cache=True,
                                        num_beams=1,
                                        tokenizer=tokenizer)
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)